Notebook Analysis Rubric #1:

Describe in your writeup (and identify where in your code) how you modified or added functions to add obstacle and rock sample identification:

Obstacle detection was rather easy, to do this I defined the "complement" of the navigable terrain; that is, instead of definig a threshold of RGB > 160 using bitwise '&' I defined a threshold of RGB < 160 as follows:

In [ ]:
def color_thresh3(img):
    color_select = np.zeros_like(img[:,:,:])
    in_obstacle_thresh = (img[:,:,0] < 160) \
                & (img[:,:,1] < 160) \
                & (img[:,:,2] < 160)
    color_select[in_obstacle_thresh] = 255       
    return color_select

To identify yellow rocks I fiddled around with an RGB color palette window on the internet and determined approximate RGB bounds for a yellow color.  (It turned out this worked for the notebook but did NOT work well enough for the autonomous rover, so I later modified it again):

In [ ]:
def color_thresh2(img):
    color_select = np.zeros_like(img[:,:,:])
    in_rock_thresh = (img[:,:,2]<50) \
    & (img[:,:,0] > 100) & (img[:,:,1] > 100)
    color_select[in_rock_thresh] = 255
    return color_select

OH, and by the way you will notice that in all 3 versions of color_thresh# I modified the original code from "np.zeros_like(img[:,:,0] to [;, ;, ;].  Later on I had to introduce a dummy variable "z_pixel" into the function rover_coords.  This was done because the video function would not map a 2-dimensional array into its output.

I then created three separate matrices with color thresholds as follows:

In [ ]:
threshed = color_thresh(warped)
threshed_yellow = color_thresh2(warped)
threshed_obstacle = color_thresh3(warped)

I then added the following codes which calculated and mapped the "threshed_obstacle" and "threshed_yellow" matrices to the worldmap (this was done in process_image so is also a partial answer to the next rubric):

In [ ]:
 # Next, Obstacle detector
    xpix, ypix, zpix = rover_coords(threshed_obstacle)
    xpos = data.xpos[data.count-1]
    ypos = data.ypos[data.count-1]
    yaw = data.yaw[data.count-1]
    scale = 10
    world_size = 200*200
    x_pix_world, y_pix_world = pix_to_world(xpix, ypix, xpos, ypos, yaw, world_size, scale)
    data.worldmap[y_pix_world, x_pix_world, 0] = 255
        # Next, Yellow Rock Detector
    xpix, ypix, zpix = rover_coords(threshed_yellow)
    xpos = data.xpos[data.count-1]
    ypos = data.ypos[data.count-1]
    yaw = data.yaw[data.count-1]
    scale = 10
    world_size = 200*200    
    x_pix_world, y_pix_world = pix_to_world(xpix, ypix, xpos, ypos, yaw, world_size, scale)
    data.worldmap[y_pix_world, x_pix_world, 1] = 255

Notebook Analysis Rubric #2:

Describe in your writeup how you modified the process_image() to demonstrate your analysis and how you created a worldmap. Include your video output with your submission:

I changed the coordinates for the output image to accommodate the navigable warped and color thresholded image in the upper right hand corner, the obstacle warped and color thresholded image in the 2nd row left, and the yellow rock detection warped and color thresholded image in the 2nd row right.  This was a matter of using simple math and multiplying the x and y coordinates by the appropriate quantities:

In [ ]:
 # Add the warped and color thresholded image in the upper right hand corner
    output_image[0:img.shape[0], img.shape[1]:] = threshed
       # Add the obtacle warped and color thresholded images in the 2nd row, left
    output_image[img.shape[0]:2*img.shape[0], 0:img.shape[1]] = threshed_obstacle
        # Add the yellow target (rock) detector images in the 2nd row, right
    output_image[img.shape[0]:2*img.shape[0], img.shape[1]:img.shape[1]*2] = threshed_yellow
    

Autonomous Navigation and Mapping:
Rubric #1:

Perception_step() and decision_step() functions have been filled in and their functionality explained in the writeup:

Several modifications were made to perception.py to optimize its function:

1) The threshold for detecting yellow rocks was fine-tuned by importing an image of a rock into "GIMP" and using the "eyedropper" tool to find the RGB boundaries which were included in the new function:

In [ ]:
def color_thresh_rock(img):
    # Create an array of zeros same xy size as img, but single channel
    color_select = np.zeros_like(img[:,:,0])
    # Require that each pixel be above all three threshold values in RGB
    # above_thresh will now contain a boolean array with "True"
    # where threshold was met
    above_thresh = (img[:,:,2]<25) \
    & (img[:,:,0] > 140) & (img[:,:,0] <205) & (img[:,:,1] > 110) & (img[:,:,1] < 180)
    # Index the array of zeros with the boolean array and set to 1
    color_select[above_thresh] = 1
    # Return the binary image
    return color_select

2) Next rotate_pix and translate_pix were completed to perform the rotation and translation of rover-centric coordinates to world coordinates:

In [ ]:
# Define a function to apply a rotation to pixel positions
def rotate_pix(xpix, ypix, yaw):
    # TODO:
    yaw_rad = yaw * np.pi/180
    # Apply a rotation
    xpix_rotated = (xpix * np.cos(yaw_rad)) - (ypix * np.sin(yaw_rad))
    ypix_rotated = (xpix * np.sin(yaw_rad)) + (ypix * np.cos(yaw_rad))
    # Return the result  
    return xpix_rotated, ypix_rotated

# Define a function to perform a translation
def translate_pix(xpix_rot, ypix_rot, xpos, ypos, scale): 
    # TODO:
    # Apply a scaling and a translation
    xpix_translated = (xpix_rot / scale) + xpos
    ypix_translated = (ypix_rot / scale) + ypos
    # Return the result  
    return xpix_translated, ypix_translated

perception_step() was filled in so that source, destination points were defined to create a perspective transform.  This perspective transform was then applied to the current rover camera image, which subsequently was thresholded in the three different ways (navigable terrain, obtacle terrain, rock identification).  Because of the "color_select[above_thres] = 1" step in each of the color_thresh_xx functions, the result was a binary image which therefore needed to be multiplied by 255 in order to display full color for each particular channel:

In [ ]:
 dst_size = 5 
# Set a bottom offset to account for the fact that the bottom of the image 
# is not the position of the rover but a bit in front of it
    bottom_offset = 6
    source = np.float32([[14, 140], [301 ,140],[200, 96], [118, 96]])
    destination = np.float32([[Rover.img.shape[1]/2 - dst_size, Rover.img.shape[0] - bottom_offset],
                  [Rover.img.shape[1]/2 + dst_size, Rover.img.shape[0] - bottom_offset],
                  [Rover.img.shape[1]/2 + dst_size, Rover.img.shape[0] - 2*dst_size - bottom_offset], 
                  [Rover.img.shape[1]/2 - dst_size, Rover.img.shape[0] - 2*dst_size - bottom_offset],
                  ])
    
    # 2) Apply perspective transform
    bin_im = perspect_transform(Rover.img, source, destination)
    # 3) Apply color threshold to identify navigable terrain/obstacles/rock samples
    bin_nav = color_thresh_nav(bin_im)
    bin_obs = color_thresh_obs(bin_im)
    bin_rock = color_thresh_rock(bin_im)
    Rover.vision_image[:,:,0] = bin_obs
    Rover.vision_image[:,:,1] = bin_rock
    Rover.vision_image[:,:,2] = bin_nav
    Rover.vision_image = Rover.vision_image*255

Next, each of the matrices is transformed into worldcentric coordinates by first converting map image pixel values to rover-centric coordinates (through a rotation around the origin of the rover) then using a translation and dilation (scale = 1/10) to convert the rover-centric pixel values to world coordinates.  This is done for EACH of the three matrices (navigable, obstacle, rock) to create three different layers of the rover.worldmap:

In [ ]:
 # 5) Convert map image pixel values to rover-centric coords
    xpix_n, ypix_n = rover_coords(bin_nav)
    xpix_o, ypix_o = rover_coords(bin_obs)
    xpix_r, ypix_r = rover_coords(bin_rock)
    # 6) Convert rover-centric pixel values to world coordinates
    scale = 10
    world_size = 200
    xpos, ypos = Rover.pos
    print ('xpos is ', xpos, 'ypos is ', ypos)
    yaw = Rover.yaw
    navigable_x_world, navigable_y_world = pix_to_world(xpix_n, ypix_n, xpos, ypos, yaw, world_size, scale)
    obstacle_x_world, obstacle_y_world = pix_to_world(xpix_o, ypix_o, xpos, ypos, yaw, world_size, scale)
    rock_x_world, rock_y_world = pix_to_world(xpix_r, ypix_r, xpos, ypos, yaw, world_size, scale)
    

... and then the Rover worldmap is updated:

In [ ]:
Rover.worldmap[obstacle_y_world, obstacle_x_world, 0] += 1
Rover.worldmap[rock_y_world, rock_x_world, 1] += 1
Rover.worldmap[navigable_y_world, navigable_x_world, 2] += 1

Decision_step() function changes and their rationale.  Many of the changes made to decision_step() were trial and error.  For example, the rover did not seem to be stopping in time (it would keep hitting obstacles), so I thought that maybe sooner braking was needed.  There fore I added "100" to line 31 in order to start the braking earlier:

In [ ]:
 elif len(Rover.nav_angles) < Rover.stop_forward + 100:

This modification (above) seemed to fix the problem of the rover running into walls or obstacles.  Another problem/issue was that the rover would start running into circles when it encountered a large open space.  To try to avoid this I considered adding a random element to the clip function in order to create some "jitter" in the steering to prevent it from getting stuck in loops.  Finally, I determined that based on the shape of the worldmap terrain and the results of the trial run it would be an advantage if the rover had a bias towards turning right, so I set up the clipping to favor a higher negative value (clockwise = right turn) see line #62:

In [ ]:
  Rover.steer = np.clip(np.mean(Rover.nav_angles * 180/np.pi), np.random.uniform(-75, -35), np.random.uniform(15, 35))

Autonomous Navigation and Mapping:
Rubric #2:

By running drive_rover.pyand launching the simulator in autonomous mode, your rover does a reasonably good job at mapping the environment.

With the changes made, the rover generally achieved a 70% fidelity and a mapping of 70% of the environment.  It also correctly identified at least 3-4 rock samples ON THE MAP.

The settings used by the simulator were:
Screen resolution: 1024 x 768
Graphics quality: Good
Select monitor: Display 1
Windowed checked
OS: Windows 10

The required video output from the notebook run is in the output folder labeled  test_mapping2.

The final notebook submission is called Notebook_2b.  Please also note that in order to get the notebook video to compile and run correctly, the notebook must be called at the command line as:

jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000
